<a href="https://colab.research.google.com/github/mbastian2/WHW_QbsDropback/blob/main/Week_WHWQBDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Install/Import/Download - RUN ONCE

In [ ]:
!pip install --upgrade nfl_data_py
!pip install -U kaleido
import nfl_data_py as nfl
import plotly.graph_objects as go
import pandas as pd
pd.set_option('display.max_columns', None)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 39.6 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.10.1 requires pandas<2.2.3dev0,>=2.0, but you have pandas 1.5.3 which is incompatible.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 1.5.3 which is incompatible.
mizani 0.13.0 requires pandas>=2.2.0, but you have pandas 1.5.3 which is incompatible.
plotnine 0.14.1 requires pandas>=2.2.0, but you have pandas 1.5.3 which is incompatible.
xarray 2024.10.0 requires pandas>=2.1, but you have pandas 1.5.3 which i

#Choose Week - Change for desired week

In [ ]:
selected_week = 11 #Replace Week

#Merge - Rerun for each week

In [ ]:
desc = nfl.import_team_desc()
ids = nfl.import_ids()
pbp = nfl.import_pbp_data([2024], downcast=True)
pbp = pbp[pbp['week'] == selected_week]
ftn = nfl.import_ftn_data([2024], downcast=True)
ftn = ftn[ftn['week'] == selected_week]
season_roster = nfl.import_seasonal_rosters([2024])
pbp = pbp[pbp['qb_dropback'] == 1.0]
pbp = pbp[pbp['two_point_attempt'] == 0]
pbp = pd.merge(pbp, season_roster[['player_id', 'player_name']], left_on='passer_id', right_on='player_id')
pbp = pd.merge(pbp, desc[['team_abbr', 'team_color', 'team_color2']], left_on='posteam', right_on='team_abbr')
pbp = pd.merge(pbp, ftn[['nflverse_game_id', 'nflverse_play_id', 'is_drop', 'is_qb_fault_sack', 'is_throw_away', 'is_interception_worthy', 'is_catchable_ball']],
               left_on=['play_id', 'game_id'], right_on=['nflverse_play_id', 'nflverse_game_id'], how='inner')

all_qbs = pbp[pbp['qb_dropback'] == 1.0].groupby('player_name').filter(lambda x: len(x) > 10)

qb_rankings = pd.DataFrame()
for qb_name in all_qbs['player_name'].unique():
    qb = all_qbs[all_qbs['player_name'] == qb_name]
    df = pd.DataFrame({'Passer': [qb_name]})
    df['Passer'] = qb_name
    df.loc[0, 'Dropbacks'] = (qb['qb_dropback'] == 1.0).sum()
    df.loc[0, 'Sacks'] = (qb['sack'] == 1.0).sum()
    df.loc[0, 'Touchdowns'] = ((qb['touchdown'] == 1.0) & (qb['fixed_drive_result'] == 'Touchdown')).sum()
    df.loc[0, 'Scrambles'] = (qb['qb_scramble'] == 1.0).sum()
    df.loc[0, 'Interceptions'] = (qb['interception'] == 1.0).sum()
    df.loc[0, 'Complete Pass'] = (qb['complete_pass'] == 1.0).sum()
    df.loc[0, 'Incomplete Pass'] = (qb['incomplete_pass'] == 1.0).sum()
    df.loc[0, 'Negative/No Gain'] = ((qb['complete_pass'] == 1.0) & (qb['yards_gained'] <= 0)).sum()
    df.loc[0, 'Short Gain'] = ((qb['complete_pass'] == 1.0) & (qb['yards_gained'] > 0) & (qb['yards_gained'] < 10)).sum()
    df.loc[0, 'Medium Gain'] = ((qb['complete_pass'] == 1.0) & (qb['yards_gained'] >= 10) & (qb['yards_gained'] < 20)).sum()
    df.loc[0, 'Long Gain'] = ((qb['complete_pass'] == 1.0) & (qb['yards_gained'] >= 20)).sum()
    df.loc[0, 'Throwaway'] = ((qb['incomplete_pass'] == 1.0) & (qb['is_throw_away'] == 1.0)).sum()
    df.loc[0, 'Drop'] = ((qb['incomplete_pass'] == 1.0) & (qb['is_drop'] == 1.0)).sum()
    df.loc[0, 'Uncatchable'] = ((qb['incomplete_pass'] == 1.0) & (qb['is_catchable_ball'] == False)).sum()
    df.loc[0, 'PBU'] = ((qb['incomplete_pass'] == 1.0) & (qb['pass_defense_1_player_id'].isna()) & (qb['is_drop'] == 0.0) & (qb['is_catchable_ball'] == True)).sum()
    df.loc[0, 'QB Int'] = (((qb['interception'] == 1.0) & qb['is_interception_worthy'] == 1.0)).sum()
    df.loc[0, 'Short Scramble'] = ((qb['qb_scramble'] == 1.0) & (qb['yards_gained'] >= 0) & (qb['yards_gained'] <= 5)).sum()
    df.loc[0, 'Medium Scramble'] = ((qb['qb_scramble'] == 1.0) & (qb['yards_gained'] > 5) & (qb['yards_gained'] < 15)).sum()
    df.loc[0, 'Long Scramble'] = ((qb['qb_scramble'] == 1.0) & (qb['yards_gained'] >= 15)).sum()
    df.loc[0, 'QB Sack'] = (((qb['pass_attempt'] == 1.0) & qb['is_qb_fault_sack'] == 1.0)).sum()
    df.loc[0, 'Passing Touchdown'] = ((qb['complete_pass'] == 1) & (qb['touchdown'] == 1.0) & (qb['fixed_drive_result'] == 'Touchdown')).sum()
    df.loc[0, 'Rushing Touchdown'] = ((qb['qb_scramble'] == 1) & (qb['touchdown'] == 1.0) & (qb['fixed_drive_result'] == 'Touchdown')).sum()
    df.loc[0, 'First Down'] = ((qb['first_down'] == 1.0)).sum()
    df.loc[0, 'First Down Pass'] = ((qb['first_down'] == 1.0) & (qb['first_down_pass'] == 1.0)).sum()
    df.loc[0, 'First Down Rush'] = ((qb['first_down'] == 1.0) & (qb['first_down_rush'] == 1.0)).sum()
    df.loc[0, 'First Down Penalty'] = ((qb['first_down'] == 1.0) & (qb['first_down_penalty'] == 1.0)).sum()
    df.loc[0, 'posteam'] = qb['posteam'].iloc[0]

    df['Sack %'] = (df['Sacks'] / df['Dropbacks']) * 100
    df['Td %'] = (df['Touchdowns'] / df['Dropbacks']) * 100
    df['Scr %'] = (df['Scrambles'] / df['Dropbacks']) * 100
    df['Int %'] = (df['Interceptions'] / df['Dropbacks']) * 100
    df['Comp %'] = (df['Complete Pass'] / df['Dropbacks']) * 100
    df['Inc %'] = (df['Incomplete Pass'] / df['Dropbacks']) * 100
    df['NegGain %'] = (df['Negative/No Gain'] / df['Dropbacks']) * 100
    df['Short %'] = (df['Short Gain'] / df['Dropbacks']) * 100
    df['Med %'] = (df['Medium Gain'] / df['Dropbacks']) * 100
    df['Deep %'] = (df['Long Gain'] / df['Dropbacks']) * 100
    df['Thrawy %'] = (df['Throwaway'] / df['Dropbacks']) * 100
    df['Drop %'] = (df['Drop'] / df['Dropbacks']) * 100
    df['Uncatchable %'] = (df['Uncatchable'] / df['Dropbacks']) * 100
    df['PBU %'] = (df['PBU'] / df['Dropbacks']) * 100
    df['QB Int %'] = (df['QB Int'] / df['Dropbacks']) * 100
    df['Short Scr %'] = (df['Short Scramble'] / df['Dropbacks']) * 100
    df['Med Scr %'] = (df['Medium Scramble'] / df['Dropbacks']) * 100
    df['Long Scr %'] = (df['Long Scramble'] / df['Dropbacks']) * 100
    df['QB Sack %'] = (df['QB Sack'] / df['Dropbacks']) * 100
    df['Pass Td %'] = (df['Passing Touchdown'] / df['Dropbacks']) * 100
    df['Rush Td %'] = (df['Rushing Touchdown'] / df['Dropbacks']) * 100
    df['FD %'] = (df['First Down'] / df['Dropbacks']) * 100
    df['FDPa %'] = (df['First Down Pass'] / df['Dropbacks']) * 100
    df['FDR %'] = (df['First Down Rush'] / df['Dropbacks']) * 100
    df['FDPe %'] = (df['First Down Penalty'] / df['Dropbacks']) * 100

    qb_rankings = pd.concat([qb_rankings, df], ignore_index=True)

qb_rankings = pd.merge(qb_rankings, desc[['team_abbr', 'team_color', 'team_color2']], left_on='posteam', right_on='team_abbr', how='left')
qb_rankings = qb_rankings.loc[qb_rankings.groupby('posteam')['Dropbacks'].idxmax()]

qb_rankings['Dropbacks Rank'] = qb_rankings['Dropbacks'].rank(ascending=False)
qb_rankings['Sack % Rank'] = qb_rankings['Sack %'].rank(ascending=False)
qb_rankings['Td % Rank'] = qb_rankings['Td %'].rank(ascending=False)
qb_rankings['Scr % Rank'] = qb_rankings['Scr %'].rank(ascending=False)
qb_rankings['Int % Rank'] = qb_rankings['Int %'].rank(ascending=False)
qb_rankings['Comp % Rank'] = qb_rankings['Comp %'].rank(ascending=False)
qb_rankings['Inc % Rank'] = qb_rankings['Inc %'].rank(ascending=False)
qb_rankings['NegGain % Rank'] = qb_rankings['NegGain %'].rank(ascending=False)
qb_rankings['Short % Rank'] = qb_rankings['Short %'].rank(ascending=False)
qb_rankings['Med % Rank'] = qb_rankings['Med %'].rank(ascending=False)
qb_rankings['Deep % Rank'] = qb_rankings['Deep %'].rank(ascending=False)
qb_rankings['Thrawy % Rank'] = qb_rankings['Thrawy %'].rank(ascending=False)
qb_rankings['Drop % Rank'] = qb_rankings['Drop %'].rank(ascending=False)
qb_rankings['Uncatchable % Rank'] = qb_rankings['Uncatchable %'].rank(ascending=False)
qb_rankings['PBU % Rank'] = qb_rankings['PBU %'].rank(ascending=False)
qb_rankings['QB Int % Rank'] = qb_rankings['QB Int %'].rank(ascending=False)
qb_rankings['Short Scr % Rank'] = qb_rankings['Short Scr %'].rank(ascending=False)
qb_rankings['Med Scr % Rank'] = qb_rankings['Med Scr %'].rank(ascending=False)
qb_rankings['Long Scr % Rank'] = qb_rankings['Long Scr %'].rank(ascending=False)
qb_rankings['QB Sack % Rank'] = qb_rankings['QB Sack %'].rank(ascending=False)
qb_rankings['Pass Td % Rank'] = qb_rankings['Pass Td %'].rank(ascending=False)
qb_rankings['Rush Td % Rank'] = qb_rankings['Rush Td %'].rank(ascending=False)
qb_rankings['FD % Rank'] = qb_rankings['FD %'].rank(ascending=False)
qb_rankings['FDPa % Rank'] = qb_rankings['FDPa %'].rank(ascending=False)
qb_rankings['FDR % Rank'] = qb_rankings['FDR %'].rank(ascending=False)
qb_rankings['FDPe % Rank'] = qb_rankings['FDPe %'].rank(ascending=False)

df = qb_rankings
count = df.shape[0]

2024 done.
Downcasting floats.
Downcasting floats.


#Choose QB - Rerun with each QB


In [ ]:
qb = df[df['Passer'] == 'Jayden Daniels']

#Create Graph - Rerun with each QB

In [ ]:
labels = [
    f"Dropbacks {qb['Dropbacks'].iloc[0]:.0f} ({qb['Dropbacks Rank'].iloc[0]:.0f} / " + str(count) + ")",
    f"Complete Pass {qb['Comp %'].iloc[0]:.1f}% ({qb['Comp % Rank'].iloc[0]:.0f} / " + str(count) + ")",
    f"Incomplete Pass {qb['Inc %'].iloc[0]:.1f}% ({qb['Inc % Rank'].iloc[0]:.0f} / " + str(count) + ")",
    f"Interception {qb['Int %'].iloc[0]:.1f}% ({qb['Int % Rank'].iloc[0]:.0f} / " + str(count) + ")",
    f"Scramble {qb['Scr %'].iloc[0]:.1f}% ({qb['Scr % Rank'].iloc[0]:.0f} / " + str(count) + ")",
    f"Sack {qb['Sack %'].iloc[0]:.1f}% ({qb['Sack % Rank'].iloc[0]:.0f} / " + str(count) + ")",
    f"Touchdown {qb['Td %'].iloc[0]:.1f}% ({qb['Td % Rank'].iloc[0]:.0f} / " + str(count) + ")",
    f"First Down {qb['FD %'].iloc[0]:.1f}% ({qb['FD % Rank'].iloc[0]:.0f} / " + str(count) + ")",
    f"Negative/No Gain {qb['NegGain %'].iloc[0]:.1f}% ({qb['NegGain % Rank'].iloc[0]:.0f} / " + str(count) + ")",
    f"(0-10] Yard Gain {qb['Short %'].iloc[0]:.1f}% ({qb['Short % Rank'].iloc[0]:.0f} / " + str(count) + ")",
    f"(10-20) Yard Gain {qb['Med %'].iloc[0]:.1f}% ({qb['Med % Rank'].iloc[0]:.0f} / " + str(count) + ")",
    f"20+ Yard Gain {qb['Deep %'].iloc[0]:.1f}% ({qb['Deep % Rank'].iloc[0]:.0f} / " + str(count) + ")",
    f"Throwaway {qb['Thrawy %'].iloc[0]:.1f}% ({qb['Thrawy % Rank'].iloc[0]:.0f} / " + str(count) + ")",
    f"Drop {qb['Drop %'].iloc[0]:.1f}% ({qb['Drop % Rank'].iloc[0]:.0f} / " + str(count) + ")",
    f"Uncatchable {qb['Uncatchable %'].iloc[0]:.1f}% ({qb['Uncatchable % Rank'].iloc[0]:.0f} / " + str(count) + ")",
    f"PBU {qb['PBU %'].iloc[0]:.1f}% ({qb['PBU % Rank'].iloc[0]:.0f} / " + str(count) + ")",
    f"QB Fault Int {qb['QB Int %'].iloc[0]:.1f}% ({qb['QB Int % Rank'].iloc[0]:.0f} / " + str(count) + ")",
    f"[0-5] Yard Gain {qb['Short Scr %'].iloc[0]:.1f}% ({qb['Short Scr % Rank'].iloc[0]:.0f} / " + str(count) + ")",
    f"(5-10) Gain {qb['Med Scr %'].iloc[0]:.1f}% ({qb['Med Scr % Rank'].iloc[0]:.0f} / " + str(count) + ")",
    f"10+ Yard Gain {qb['Long Scr %'].iloc[0]:.1f}% ({qb['Long Scr % Rank'].iloc[0]:.0f} / " + str(count) + ")",
    f"QB Fault Sack {qb['QB Sack %'].iloc[0]:.1f}% ({qb['QB Sack % Rank'].iloc[0]:.0f} / " + str(count) + ")",
    f"Passing Touchdown {qb['Pass Td %'].iloc[0]:.1f}% ({qb['Pass Td % Rank'].iloc[0]:.0f} / " + str(count) + ")",
    f"Rushing Touchdown {qb['Rush Td %'].iloc[0]:.1f}% ({qb['Rush Td % Rank'].iloc[0]:.0f} / " + str(count) + ")",
    f"First Down Pass {qb['FDPa %'].iloc[0]:.1f}% ({qb['FDPa % Rank'].iloc[0]:.0f} / " + str(count) + ")",
    f"First Down Scramble {qb['FDR %'].iloc[0]:.1f}% ({qb['FDR % Rank'].iloc[0]:.0f} / " + str(count) + ")",
    f"First Down Penalty {qb['FDPe %'].iloc[0]:.1f}% ({qb['FDPe % Rank'].iloc[0]:.0f} / " + str(count) + ")"
]

source = [0,0,0,0,0,0,0, 1,1,1,1, 2,2,2,2, 3, 4,4,4, 5, 6,6, 7,7,7]
target = [1,2,3,4,5,6,7, 8,9,10,11 ,12,13,14,15, 16, 17,18,19, 20, 21,22, 23,24,25]
values = [qb['Complete Pass'].iloc[0], qb['Incomplete Pass'].iloc[0], qb['Interceptions'].iloc[0], qb['Scrambles'].iloc[0], qb['Sacks'].iloc[0], qb['Touchdowns'].iloc[0], qb['First Down'].iloc[0],
          qb['Negative/No Gain'].iloc[0], qb['Short Gain'].iloc[0], qb['Medium Gain'].iloc[0], qb['Long Gain'].iloc[0], qb['Throwaway'].iloc[0], qb['Drop'].iloc[0], qb['Uncatchable'].iloc[0],
          qb['PBU'].iloc[0], qb['QB Int'].iloc[0], qb['Short Scramble'].iloc[0], qb['Medium Scramble'].iloc[0], qb['Long Scramble'].iloc[0], qb['QB Sack'].iloc[0], qb['Passing Touchdown'].iloc[0],
          qb['Rushing Touchdown'].iloc[0], qb['First Down Pass'].iloc[0], qb['First Down Rush'].iloc[0], qb['First Down Penalty'].iloc[0]]
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=50,
        thickness=20,
        label=labels,
        color = qb['team_color'].iloc[0]
),
    link=dict(
        source=source,
        target=target,
        value=values,
        color = qb['team_color2'].iloc[0],
        line=dict(color='gainsboro', width=0.5)
    )
)])
fig.update_layout(
    width = 1000,
    height = 600,
    title_text=f"What Happened When {qb['Passer'].iloc[0]} Dropped Back In Week {selected_week}?",
    title_font=dict(size=16, color='black'),
    title_xanchor='center',
    title_x=0.5,
    title_y=0.9,
    title_yanchor='top',
    title_pad=dict(b=10),
    margin=dict(l=20, r=20, t=20, b=20),
)
fig.add_annotation(
    text="X:@MB_NFL, Data: FTN and nfl_data_py",
    xref="paper", yref="paper",
    x=0.05, y=0.05,
    showarrow=False,
    font=dict(size=12, color="black"),
    bgcolor="white",
    bordercolor="black",
    borderwidth=1,
    borderpad=4,
)

#Show Graph - Rerun with each QB

In [ ]:
fig.show()